In [ ]:
import numpy as np
import pandas as pd
import xgboost as xgb
xgb_reg = xgb.XGBRegressor()
xgb_reg.load_model('./xgb.ubj')

In [ ]:
def gen_formula(n):

    f0 = np.round(np.random.uniform(300, 1000, n), 1).reshape(n,1)
    f1 = np.round(np.random.uniform(0, 1000, n), 1).reshape(n,1)
    f5 = np.round(np.random.uniform(0, 1000, n), 1).reshape(n,1)
    f6 = np.round(np.random.uniform(0, 1000, n), 1).reshape(n,1)
    f7 = np.round(np.random.uniform(0, 1000, n), 1).reshape(n,1)

    f2 = np.tile([[275]],(n,1))
    f3 = np.tile([[125]],(n,1))
    f4 = 2*f5
    f8 = np.tile([[23]],(n,1))
    f9 = np.tile([[156]],(n,1))
    
    initial_formula = pd.DataFrame(np.concatenate([f0,f1,f2,f3,f4,f5,f6,f7,f8,f9],axis=1),columns=['f0','f1','f2','f3','f4','f5','f6','f7','f8','f9'])
    
    return initial_formula
    

def select_formula(initial_formula):
    
    condition1 = ((initial_formula['f1']/600) / (initial_formula['f1']/600 + initial_formula['f0']/3150)).round(1)  # [0,0.6]
    condition2 = ((initial_formula['f6']/650) / (initial_formula['f4']/2650 + initial_formula['f5']/2710 + initial_formula['f6']/650)).round(1)  # [0,0.6]
    condition3 = (initial_formula['f7'] / (initial_formula[['f0','f1','f2','f3']].sum(axis=1))).round(2)  # [0.17,0.19]
    condition4 = initial_formula.sum(axis=1).round(0)  # [1800,1950]
    condition5 = ((initial_formula[['f0','f1','f2','f3']].sum(axis=1)) / (initial_formula[['f4','f5','f6']].sum(axis=1))).round(1)  # [1.1,1.4]
    condition6 = (initial_formula['f0']/3150 + initial_formula['f1']/600 + initial_formula['f2']/2420 + initial_formula['f3']/2200 + initial_formula['f4']/2650 + initial_formula['f5']/2710 \
    + initial_formula['f6']/650 + initial_formula['f7']/1000 + initial_formula['f9']/7800).round(0)  # [0.9,1]
    
    index = np.where((condition1>0) & (condition1<=0.6) & (condition2>0) & (condition2<=0.6) & (condition3>=0.17) & (condition3<=0.19) & (condition4>=1800) & (condition4<=1950) & (condition5>=1.1) & (condition5<=1.4) & (condition6>=0.9) & (condition6<=1))
    selective_formula = initial_formula.iloc[index]
    
    return selective_formula

def main(n):

    # 取消科学计数法，并设置小数点后精度
    np.set_printoptions(suppress=True, precision=8)
    
    initial_formula = gen_formula(n)
    selective_formula = select_formula(initial_formula).values
    
    mean_ = np.expand_dims(np.mean(selective_formula,axis=1),axis=1)
    std_ = np.expand_dims(np.std(selective_formula,axis=1),axis=1)
    input_formula = (selective_formula - mean_)/std_
    
    pred_strength = xgb_reg.predict(input_formula)
    pred_strength = (pred_strength * std_[:,0] + mean_[:,0]).round(1)
    index = np.where((pred_strength >= 120) & (pred_strength <= 160))  # 强度[120,160]
    
    final_formula = selective_formula[index]
    final_strength = pred_strength[index]

    return final_formula, np.expand_dims(final_strength,axis=1)

In [ ]:
columns = ['C', 'LWP', 'SF', 'SCMs', 'QS', 'QP', 'LWA', 'W', 'SP', 'F', 'CS', 'TD','SEF']
df = pd.DataFrame(columns=columns)
for i in range(50):
    
    final_formula, final_strength = main(n=1000000)
    final_dense = np.expand_dims(np.sum(final_formula,axis=1),axis=1)
    SEF = (final_strength/final_dense*1000).round(0)
    results = np.concatenate([final_formula,final_strength,final_dense,SEF],axis=1)
    df1 = pd.DataFrame(results,columns=columns)
    df = pd.concat([df,df1],axis=0, ignore_index=True)
df

In [ ]:
df.to_excel('./predicted_mix.xlsx')